In [20]:
import pandas as pd
import numpy as np
import sys
import nbimporter
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vectors

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

#%load_ext autotime


In [2]:
#####RDD Function Area######
def print_rdd(rdd):
    for x in rdd.collect():
        print x
        
#create pyspark dataframe from rdd
def df_from_rdd(rdd):
    pass

#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df


In [3]:
####FUNCTION AREA###########

#get workout info for workoutid
def get_rdd_workout_from_workoutid(workoutid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['workoutid']==workoutid)
    return filtered_df
    
'''
##given user id, returns dataframe of workouts by that user
##NOT DONE
def get_workouts_for_userid(userid, df_workout):
    #get workoutIds done by this user
    workout_ids = df_workout\
        .filter(df_workout['userid']==userid)\
        .select('workoutid').flatMap(lambda x:x).collect()
    print workout_ids
    df = df_sport.where(col("workoutid").isin(workout_ids))
    print df.show()
    print df.count()
    return userid
'''

#get workoutids for user
def get_rdd_workoutids_for_userid(userid, df_workouts):
    filtered_df = df_workouts.filter(df_workouts['id']==userid)
    workoutids = filtered_df.select('workoutid').flatMap(list)
    return workoutids

#convert type of a column of a df
def convertColumn(df, name, new_type):
    df_1 = df.withColumnRenamed(name, "swap")
    return df_1.withColumn(name, df_1["swap"].cast(new_type)).drop("swap")



# #fills 
# def fill_na_with_values(dict){
    
# }

In [4]:
url = 'postgresql://34.199.246.37:5432/endomondo'
properties = {'user':'endomondo', 'password':'End0m0ndo'}

df_workouts = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr', properties=properties
)

df_users = DataFrameReader(sqlContext).jdbc(
   url='jdbc:%s' % url, table='public.run_sub_with_hr_users', properties=properties
)



In [19]:
print type(df_workouts)
print type(df_users)
df_users = df_users.head(2)


<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.execution.EvaluatePython.takeAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, localhost): java.lang.NullPointerException
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$$anon$1.getNext(JDBCRDD.scala:442)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$$anon$1.hasNext(JDBCRDD.scala:518)
	at scala.collection.Iterator$$anon$14.hasNext(Iterator.scala:388)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:327)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:308)
	at scala.collection.Iterator$class.foreach(Iterator.scala:727)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1157)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:48)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:103)
	at scala.

In [18]:
df_workouts.toPandas()

,time,altitude,heart_rate,latitude,longitude,speed,workoutid,id
0,1433852797,None,156.00000,55.7094071000,37.7599174000,None,538958939,1188410
1,1433852805,None,155.00000,55.7094071000,37.7599174000,None,538958939,1188411
2,1433852810,None,155.00000,55.7095247000,37.7596090000,None,538958939,1188412
3,1433852812,None,159.00000,55.7095408000,37.7595179000,None,538958939,1188413
4,1433852815,None,159.00000,55.7095773000,37.7593918000,None,538958939,1188414
5,1433852821,None,156.00000,55.7096140000,37.7591375000,None,538958939,1188415
6,1433852825,None,155.00000,55.7096398000,37.7590429000,None,538958939,1188416
7,1433852828,None,155.00000,55.7096398000,37.7590429000,None,538958939,1188417
8,1433682342,None,102.00000,55.7094226000,37.7584615000,None,537562356,1188418
9,1433682342,None,102.00000,55.7094226000,37.7584615000,None,537562356,1188419


In [ ]:
print df_workouts.count()
print df_users.count()

In [ ]:
wout_ids = get_rdd_workoutids_for_userid(1295505, df_workouts).collect()

#get one workout id from the list
wout_id = wout_ids[0]
print wout_id

#get time series data for that workout
df_workouts_for_user = get_rdd_workout_from_workoutid(wout_id, df_workouts)
    
print df_workouts_for_user.count()

In [ ]:
#fill nans with a value
df4 = df_workouts_for_user.na.fill(0)

In [ ]:
#make each row in the dataframe into feature vector for clustering

vecAssembler = VectorAssembler(inputCols=['altitude',\
 'heart_rate',\
 'latitude',\
 'longitude',\
 'speed'], outputCol="features")
final_df = vecAssembler.transform(df4)
final_df = final_df.select('features')


In [ ]:
#make rows into "dense vectors" for clustering
final_df1 = final_df.rdd\
    .map(lambda row : Vectors.dense([item for item in row]))
final_df1.collect()

In [ ]:
# Build the model (cluster the data. For now only clusteres on datapoints)
clusters = KMeans.train(final_df1, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
# def error(point):
#     center = clusters.centers[clusters.predict(point)]
#     return sqrt(sum([x**2 for x in (point - center)]))

# WSSSE = final_df1.map(lambda point: error(point)).reduce(lambda x, y: x + y)
# print("Within Set Sum of Squared Error = " + str(WSSSE))
# Save and load model
#clusters.save(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")
#sameModel = KMeansModel.load(sc, "target/org/apache/spark/PythonKMeansExample/KMeansModel")

In [ ]:
clusters.clusterCenters